# Pump it up - Data mining the water table
#### https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

### Balazs Balogh - 01. 2020.

In [1]:
"""
Import the packages.
"""

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.experimental import enable_hist_gradient_boosting  # explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingClassifier # now you can import normally from ensemble

from sklearn import tree, ensemble, metrics, svm
from xgboost import *
from xgboost import XGBClassifier

from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.utils import class_weight

from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

pd.set_option('display.max_columns', 50)

In [2]:
data_selected = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/workshops/master/10_pump_it_up_2/pumpitup_train_preporcessed.csv')
test_selected = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/workshops/master/10_pump_it_up_2/pumpitup_test_preprocessed.csv')

In [3]:
target = data_selected['label']
features = data_selected.drop('label', axis=1)

In [4]:
"""
Creating the train / test / validation datasets.
"""

X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, stratify=target, random_state=123)
print('X_train original shape:', X_train.shape)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.85, stratify=y_train, random_state=123)
print('X_train shape after creating validation set:', X_train.shape)
print('X_val shape:', X_val.shape)

X_train original shape: (47520, 69)
X_train shape after creating validation set: (40392, 69)
X_val shape: (7128, 69)


In [5]:
"""
Oversampling example with SMOTE

https://towardsdatascience.com/sampling-techniques-for-extremely-imbalanced-data-part-ii-over-sampling-d61b43bc4879
https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets
"""

from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

print('Original data:', sorted(Counter(y_train).items()))
print('Resampled data:', sorted(Counter(y_resampled).items()))

Using TensorFlow backend.


Original data: [(0, 15520), (1, 2936), (2, 21936)]
Resampled data: [(0, 21936), (1, 21936), (2, 21936)]


# Modeling

### Experimenting with PCA

### XGBoost

In [6]:
"""
"""

xgboost_classifier = XGBClassifier(eta=0.075, max_depth=3, min_child_weight=0, n_estimators=200, scale_pos_weigth=5,
                                   objective='multi:softmax', eval_metric='merror', num_class=3)

xgboost = xgboost_classifier.fit(X_train, y_train.values.ravel())
# xgboost = xgboost_classifier.fit(X_resampled, y_resampled.ravel()) # With SMOTE oversampling data

expected = y_test
predicted = xgboost.predict(X_test)

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.82      0.63      0.71      4565
           1       0.59      0.10      0.17       863
           2       0.72      0.91      0.81      6452

    accuracy                           0.75     11880
   macro avg       0.71      0.55      0.56     11880
weighted avg       0.75      0.75      0.72     11880



In [7]:
"""
Testing the validation set, which is unseen for the model.
"""

validation_expected = y_val
validation_predicted = xgboost.predict(X_val)

print(metrics.classification_report(validation_expected, validation_predicted))

              precision    recall  f1-score   support

           0       0.82      0.62      0.71      2739
           1       0.60      0.11      0.18       518
           2       0.72      0.92      0.80      3871

    accuracy                           0.74      7128
   macro avg       0.71      0.55      0.56      7128
weighted avg       0.75      0.74      0.72      7128



In [36]:
"""
Get feature importances.
Of course, this works without PCA, so when we know the original columns.

"The Gain is the most relevant attribute to interpret the relative importance of each feature."
"""

xgb_fi = xgboost.get_booster().get_score(importance_type="gain")

xgboost_feature_importances = pd.DataFrame(list(xgb_fi.items()), columns=['feature_name', 'importance'])

xgboost_feature_importances.sort_values(by=['importance'], ascending=False).head(10)

,feature_name,importance
0,waterpoint_type_group_other,263.982527
2,quantity_insufficient,159.144926
13,extraction_type_class_other,144.052858
1,quantity_enough,108.298299
23,quantity_seasonal,84.241811
16,construction_year_70s,68.174820
18,funder_gov,66.466003
3,amount_tsh,48.297427
32,construction_year_80s,45.533163
28,extraction_type_class_motorpump,41.511333


In [ ]:
"""
GridSearchCV, the most computation expensive part of the process. With a relatively small amount of parameters, it lifted
the performance from 73.9% to 75.2%.
"""

xgboost = XGBClassifier(objective='multi:softmax', eval_metric='merror', num_class=3, n_jobs=-1)

params = {'n_estimators': [100, 200, 300],
          'max_depth': [5, 8, 15],
          'learning_rate': [0.01, 0.075, 0.1, 0.2],
          'min_child_weight': [3, 5, 7]
         }

grid_search = GridSearchCV(estimator=xgboost, cv=4, param_grid=params, n_jobs=-1, verbose=10)

grid_search.fit(X_train, y_train.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
"""
Cross validation took about 5 minutes, and it showed a 74% accuracy. We can see each fold, the model is straight, because
it keeps bringing the same result.
"""

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

scores = cross_val_score(xgboost, X_train, y_train, cv=cv, verbose=10)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Random Forest

In [37]:
clf = ensemble.RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=2)
clf = clf.fit(X_train, y_train)

expected = y_test
predicted = clf.predict(X_test)

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.85      0.61      0.71      4565
           1       0.56      0.06      0.10       863
           2       0.71      0.94      0.81      6452

    accuracy                           0.75     11880
   macro avg       0.71      0.54      0.54     11880
weighted avg       0.75      0.75      0.72     11880



In [ ]:
params = {'n_estimators' : [10, 50, 200],
          'min_samples_split' : [2, 3, 4],
          'max_depth': [3, 5, 8, 10]}

grid_search = GridSearchCV(estimator=clf, cv=3, param_grid=params, n_jobs=-1 verbose=10)

grid_search.fit(X_train, y_train.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

### SVM

In [ ]:
"""
SVM clearly not the best fit for this problem, because of the 100+ columns.
"""

svc = svm.SVC()
svc = svc.fit(X_train, y_train)

expected = y_test
predicted = svc.predict(X_test)

print(metrics.classification_report(expected, predicted))

### Logistic regression

In [38]:
"""
It doesn't perform well.
"""

logreg = LogisticRegression(multi_class='auto', solver='newton-cg', max_iter=50, n_jobs=-1)
logreg.fit(X_train, y_train)

expected = y_test
predicted = logreg.predict(X_test)

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.78      0.61      0.68      4565
           1       0.26      0.01      0.02       863
           2       0.70      0.90      0.79      6452

    accuracy                           0.72     11880
   macro avg       0.58      0.51      0.50     11880
weighted avg       0.70      0.72      0.69     11880



In [ ]:
"""
GridSearch
"""

params = {'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
          'max_iter': [50, 150, 200]
         }

grid_search = GridSearchCV(estimator=logreg, cv=5, param_grid=params, n_jobs=-1 verbose=10)

grid_search.fit(X_train, y_train.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

### Gradient Boosting Classifier

In [39]:
"""
This was the best model, scored 80.11% on the DrivenData submission. It is really slow though.
"""

gradient = GradientBoostingClassifier(n_estimators=100, learning_rate=0.075, max_depth=14,
                                     min_samples_leaf=16, verbose=5)

gradient = gradient.fit(X_train, y_train)

expected = y_test
predicted = gradient.predict(X_test)

print(metrics.classification_report(expected, predicted))

      Iter       Train Loss   Remaining Time 
         1       33369.2148           16.46m
         2       31287.2605           17.02m
         3       29545.9655           18.20m
         4       28040.9708           17.90m
         5       26702.7998           17.82m
         6       25532.2280           17.96m
         7       24497.6548           18.12m
         8       23572.8610           17.89m
         9       22752.6387           17.97m
        10       21987.2140           17.67m
        11       21311.4331           17.65m
        12       20687.4989           17.48m
        13       20126.3307           16.93m
        14       19621.3449           16.46m
        15       19155.8965           15.94m
        16       18689.5576           15.58m
        17       18282.6314           15.29m
        18       17880.4935           15.00m
        19       17548.9996           14.70m
        20       17224.4807           14.70m
        21       16928.6945           14.68m
        2

In [40]:
"""
Testing the validation set, which is unseen for the model.
"""

validation_expected = y_val
validation_predicted = gradient.predict(X_val)

print(metrics.classification_report(validation_expected, validation_predicted))

              precision    recall  f1-score   support

           0       0.84      0.76      0.79      2739
           1       0.52      0.27      0.36       518
           2       0.79      0.89      0.84      3871

    accuracy                           0.79      7128
   macro avg       0.71      0.64      0.66      7128
weighted avg       0.79      0.79      0.79      7128



In [ ]:
"""
RandomizedSearchCV, to make it faster than GridSearchCV.
"""
gradient = GradientBoostingClassifier(verbose=5)

params = {'n_estimators': [100, 150, 200, 250, 300],
          'max_depth': [5, 8, 10, 15, 20],
          'learning_rate': [0.075, 0.01, 0.05, 0.1],
          'min_samples_leaf': [10, 15, 20, 25]
         }

n_iter_search = 10
random_search = RandomizedSearchCV(gradient, param_distributions=params,
                                   n_iter=n_iter_search, n_jobs=-1, cv=3, verbose=11)

random_search.fit(X_train, y_train.values.ravel())

print(random_search.best_score_)
print(random_search.best_params_)

### Hist Gradient Boosing Classifier

In [41]:
"""
Histogram-based Gradient Boosting Classification Tree.

This estimator is much faster than GradientBoostingClassifier for big datasets (n_samples >= 10 000).
"""

hist_gradient = HistGradientBoostingClassifier(learning_rate=0.2, loss='categorical_crossentropy', 
                                               max_depth=8, min_samples_leaf=15)

hist_gradient = hist_gradient.fit(X_train, y_train)

expected = y_test
predicted = hist_gradient.predict(X_test)

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.84      0.75      0.79      4565
           1       0.56      0.27      0.36       863
           2       0.78      0.90      0.84      6452

    accuracy                           0.79     11880
   macro avg       0.73      0.64      0.66     11880
weighted avg       0.79      0.79      0.78     11880



In [42]:
"""
Testing the validation set, which is unseen for the model.
"""

validation_expected = y_val
validation_predicted = hist_gradient.predict(X_val)

print(metrics.classification_report(validation_expected, validation_predicted))

              precision    recall  f1-score   support

           0       0.84      0.74      0.78      2739
           1       0.57      0.25      0.35       518
           2       0.77      0.90      0.83      3871

    accuracy                           0.79      7128
   macro avg       0.73      0.63      0.66      7128
weighted avg       0.79      0.79      0.78      7128



In [ ]:
"""
GridSearchCV, the most computation expensive part of the process.
"""

hist_gradient = HistGradientBoostingClassifier(verbose=2)

params = {'max_iter': [100, 150, 200],
          'loss': ['categorical_crossentropy'],
          'max_depth': [5, 8, 15],
          'learning_rate': [0.01, 0.75, 0.1, 0.2],
          'min_samples_leaf': [5, 15, 20]
         }

grid_search = GridSearchCV(estimator=hist_gradient, cv=4, param_grid=params, n_jobs=-1, verbose=10)

grid_search.fit(X_train, y_train.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

### LightGBM

In [43]:
"""
LightGBM is another fast tree based gradient boosting algorithm, which supports GPU, and parallel learning.
"""

lgbm = LGBMClassifier(objective='multiclass', learning_rate=0.1, num_iterations=200, num_leaves=50, random_state=123)

lgbm.fit(X_train, y_train)

expected = y_test
predicted = lgbm.predict(X_test)

print(metrics.classification_report(expected, predicted))

C:\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


              precision    recall  f1-score   support

           0       0.85      0.73      0.78      4565
           1       0.61      0.24      0.34       863
           2       0.77      0.91      0.84      6452

    accuracy                           0.79     11880
   macro avg       0.74      0.63      0.65     11880
weighted avg       0.79      0.79      0.78     11880



In [44]:
"""
Testing the validation set, which is unseen for the model.
"""

validation_expected = y_val
validation_predicted = lgbm.predict(X_val)

print(metrics.classification_report(validation_expected, validation_predicted))

              precision    recall  f1-score   support

           0       0.85      0.72      0.78      2739
           1       0.61      0.23      0.34       518
           2       0.77      0.91      0.83      3871

    accuracy                           0.79      7128
   macro avg       0.74      0.62      0.65      7128
weighted avg       0.79      0.79      0.78      7128



In [ ]:
"""
GridSearchCV, the most computation expensive part of the process.
"""

lgbm = LGBMClassifier(objective='multiclass', num_threads=2, verbose=2, random_state=123)

params = {'num_iterations ': [100, 150, 200],
          'max_depth': [5, 8, 15],
          'learning_rate': [0.01, 0.75, 0.1, 0.2],
          'num_leaves' : [25, 40, 50]
         }

grid_search = GridSearchCV(estimator=lgbm, cv=4, param_grid=params, n_jobs=-1, verbose=5) # n_jobs=-1 = use all the CPU cores

grid_search.fit(X_train, y_train.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

### Keras wrapper for Scikit-learn

In [45]:
"""
For imbalanced datasets like this.
"""

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights

array([0.86752577, 4.58583106, 0.61378556])

In [46]:
"""
0    15520
1     2936
2    21936

Manual class weight wasn't as good, as 'balanced'.
"It basically means replicating the smaller class until you have as many samples as in the larger one, but in an implicit way."

It needs more tuning / layers, because it stucks at 71% + set an early stopping.
"""

def model():
    
    model = Sequential()
    model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(8))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

estimator = KerasClassifier(build_fn=model, epochs=30, batch_size=5, verbose=1)
estimator.fit(X_train.values, y_train.values, class_weight=class_weights)

expected = y_test
predicted = estimator.predict(X_test)

print(metrics.classification_report(expected, predicted))

Train on 40392 samples
Epoch 1/30
40392/40392 [==============================] - 20s 485us/sample - loss: 2.2383 - accuracy: 0.5833
Epoch 2/30
40392/40392 [==============================] - 19s 474us/sample - loss: 0.9307 - accuracy: 0.6611
Epoch 3/30
40392/40392 [==============================] - 19s 460us/sample - loss: 0.8250 - accuracy: 0.6753
Epoch 4/30
40392/40392 [==============================] - 17s 417us/sample - loss: 0.7509 - accuracy: 0.6836
Epoch 5/30
40392/40392 [==============================] - 16s 408us/sample - loss: 0.7546 - accuracy: 0.6926
Epoch 6/30
40392/40392 [==============================] - 14s 350us/sample - loss: 0.7253 - accuracy: 0.6984
Epoch 7/30
40392/40392 [==============================] - 13s 318us/sample - loss: 0.7321 - accuracy: 0.7004
Epoch 8/30
40392/40392 [==============================] - 15s 361us/sample - loss: 0.7157 - accuracy: 0.7054
Epoch 9/30
40392/40392 [==============================] - 14s 344us/sample - loss: 0.7131 - accuracy: 0.7

### MLPClassifier - Neural network for scikit-learn

In [47]:
"""
It needs tuning.
"""

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='lbfgs', alpha=0.0001,
                     hidden_layer_sizes=(50, 50, 50, 50), max_iter=500, random_state=1, verbose=10)
mlp.out_activation_ = 'softmax'

mlp.fit(X_train, y_train)

expected = y_test
predicted = mlp.predict(X_test)

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.37      0.37      0.37      4565
           1       0.08      0.06      0.07       863
           2       0.53      0.56      0.55      6452

    accuracy                           0.45     11880
   macro avg       0.33      0.33      0.33     11880
weighted avg       0.44      0.45      0.44     11880



### VotingClassifier

In [49]:
"""
You can combine your best predictors as a VotingClassifier, which can enhance the performance.

26/01/2020 - The hist_gradient / lgbm / gradient combination made the best result yet: 80.35%!
"""

estimators = [('hist_gradient', hist_gradient), ('lightgbm', lgbm), ('gradient', gradient)]

ensemble = VotingClassifier(estimators, voting='hard', verbose=10)

ensemble.fit(X_train, y_train)
ensemble.score(X_test, y_test)

C:\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


      Iter       Train Loss   Remaining Time 
         1       33368.7480           14.45m
         2       31288.4000           14.12m
         3       29545.8449           13.30m
         4       28040.6424           12.77m
         5       26706.5589           12.75m
         6       25547.2156           12.50m
         7       24516.4368           12.18m
         8       23578.1669           11.97m
         9       22756.1825           11.72m
        10       21989.8351           11.58m
        11       21302.6141           11.42m
        12       20675.5112           11.39m
        13       20112.2442           11.35m
        14       19599.8916           11.24m
        15       19122.8282           11.21m
        16       18698.1707           11.05m
        17       18287.5867           10.82m
        18       17900.1732           10.61m
        19       17544.2573           10.40m
        20       17234.3065           10.25m
        21       16909.7891           10.19m
        2

0.7984848484848485

### Create the submission

In [50]:
"""
The 'status_group_nums' should be changed with the model we want to predict.
"""

submission = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/workshops/master/09_pump_it_up/SubmissionFormat.csv')
submission['status_group_nums'] = ensemble.predict(test_selected)

label_dict = {2: "functional", 1: "functional needs repair", 0 :"non functional"}
submission["status_group"] = submission["status_group_nums"].map(label_dict)
submission = submission.drop('status_group_nums', axis=1)

submission.to_csv('pumpitup.csv',index=False)

submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 2 columns):
id              14850 non-null int64
status_group    14850 non-null object
dtypes: int64(1), object(1)
memory usage: 232.2+ KB


In [ ]:
submission.head()

### Notes

In [ ]:
"""
DataCamp: Feature engineering with Python course:

One-hot and dummy encoding, drop_first=True, because if not specified the model could be distracted by duplicate info.
Numerical columns: binning with pd.cut, so if we have ten different values in a column, but three of them are the 70% of all,
the other 7 could be "other".

Scaling: StandardScaler does not affect the data's shape. This works great if your data is normally distributed 
(or closely normally distributed), an assumption that a lot of machine learning models make. Sometimes you will work 
with data that closely conforms to normality, e.g the height or weight of a population. On the other hand, many variables 
in the real world do not follow this pattern e.g, wages or age of a population. In this exercise you will use a log 
transform on the ConvertedSalary column as it has a large amount of its data centered around the lower values, 
but contains very high values also. These distributions are said to have a long right tail.
We should try the PowerTransformer for these kind of columns.

Though Tree based models won't affect by the standardization.

Fit and transform on the TRAINING data, and ONLY TRANSFORM on the TEST data. This is to avoid data leakage, because in real life
we won't have access to test data, so we have to work with what we have, when making the model. Calibrate the preprocessing
steps only on the training data. To do this in practice you train the scaler on the train set, and keep the trained scaler 
to apply it to the test set. You should never retrain a scaler on the test set. This approach is for scaling, and for removing
outilers too.

I should check the distributions of the columns to look fo outliers. Eliminate data with 3 std
(this is the more statistical approach), or the 95th quantile (column.quantile(0.95)).

More:

- VotingClassifier check. https://towardsdatascience.com/ensemble-learning-using-scikit-learn-85c4531ff86a
    74.5% but the Logistic regression, the third model may not be the best choice.

- 'construction_year' binning. Check out the earliest, and the latest dates. - This made from 75.20% to 75.25%. 
   Very small step, but helped.
   
- Dimension reduction - needs scaling first. Try with just scaling, and then with scaling and PCA. - 
  It didn't helped for first try.
  
- 'functional needs repair' oversampling - DrivenData 70.46%.

- DL solution + LightGBM

- log transformation - done, 74.55% was the driven data accuracy, on my validation set, it was 75% so it was 1% better. 

- perc_func_lga - A column I created in a past version of this notebook.

- Delete validation set, because it means 7000 rows.


"""